In [1]:
from PIL import Image
import pytesseract  # installation on windows: https://www.youtube.com/watch?v=Rb93uLXiTwA
import cv2
from pprint import pprint
import pathlib
import json
import re

In [2]:
def test_crop(file, x, y, h, w):
    img = cv2.imread(file)
    # print(img.shape)
    crop_img = img[y:y+h, x:x+w]
    cv2.imshow("cropped", crop_img)
    cv2.waitKey(0)

def crop_img(img_file, x, y, h, w):
    # !hack: need to manually tune these parameters. ok, it is only to be done once
    img = cv2.imread(str(img_file))
    print(img.shape)
    crop_img = img[y:y+h, x:x+w]
    cv2.imwrite("cropped.png", crop_img)
    # cv2.imshow("cropped", crop_img)
    # cv2.waitKey(0)
    return crop_img

def save_dict_as_json(d, json_file='data.json'):
    with open(json_file, 'w') as outfile:
        json.dump(d, outfile)

import csv
def save_dict_to_csv(out_csv, in_dict):
    with open(out_csv, 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in in_dict.items():
            writer.writerow([key, value])
            
def has_good_numbers(str_in):
    # !hack: sometimes "O" is recognised as "("
    str_in = str_in.replace("(", "0")
    regex0= "0[0-9]"
    regex1= "46"  # Sweden
    regex2= "49"  # German
    regex3= "33"  # French
    regex4= "44"  # Uk
    return any(bool(re.search(r, str_in)) for r in [regex0, regex1, regex2, regex3, regex4])

def count_numbers(str_in):
    return sum(c.isdigit() for c in str_in)

def count_alpha(str_in):
    return sum(c.isalpha() for c in str_in)

def extract_nums_names(text):
    names = []
    nums = []
    lines = text.splitlines()
    # lines is like ['name1', '', 'name2', '', '00 33 3 33.33.33 33', '', '(0033 333.33 333']
    # hence len(lines) should be divisible by 4 to form (name, num) pairs
    # hack!: just ignore the whole pack (this has to be fixed by horizontal OCR, not vertical!)
    """
    if (len(lines)+1) % 4 != 0:
        print("WARNING: inconsistent number in entries: {}".format(len(lines)))
        print(lines)
        return [], []
    """
    # issue: all entry should be separated by "". But one entry may also be ""
    # hack!: try to find two empty lines in a row
    previous_is_empty = False
    # hack!: start with name column
    name_column = True
    for line in lines:
        if not line:  # empty line
            if previous_is_empty:
                print("PATCH two NULL in a row")
                if name_column:
                    names.append("NULL")
                else:
                    nums.append("0000")
                continue
            previous_is_empty = True
        else:
            previous_is_empty = False
            # if has_good_numbers(line) and count_numbers(line) > 7:
            if count_numbers(line) > 2:  # hack!: sometimes a num is totally mis.recognized and get letters
                name_column = False
                num = ''.join(filter(lambda x: (x.isdigit() or x=="+"), line))
                nums.append(num)
            elif count_alpha(line) > 2:
                name = line.replace(" ", "_")
                name = line.replace("‘", "")
                names.append(name)
            else:
                if name_column:
                    print("  save {} as NAME".format(line))
                    names.append("NULL")
                else:
                    print("  save {} as NUM".format(line))
                    nums.append("0000")
    if len(names) != len(nums):
        print("WARNING: len(names)[{}] != len(nums)[{}]".format(len(names), len(nums)))
        print(names)
        print(nums)
        print(lines)
    return names, nums
    # !hack: the ocr go through each column independantly. Zipping is therefore dangerous

In [3]:
# before looping through all the images, define the cropping parameters
# idea is to keep only the space with nums and names
"""
x = int(1366/3.4)
y = int(768/3.2)
h = int(768/1.8)
w = int(1366/3)
"""

x = int(936/3.4)
y = int(526/3.2)
h = int(526/1.8)
w = int(936/3)
img_file = "data/example_git_hub.PNG"

test_crop(img_file, x, y, h, w)

In [4]:
counter = 0
contacts = {}

# define where the images are
data_dir = pathlib.Path.cwd()
data_dir = pathlib.Path.joinpath(data_dir, "data")

# define the image format
data_pattern = "example_git_hub.png"

# find all the images
for file in data_dir.glob(data_pattern):
    print("working on {}".format(file))
    counter += 1
    cropped_image = crop_img(file, x, y, h, w)
    img_for_orc = Image.fromarray(cropped_image)
    text = pytesseract.image_to_string(img_for_orc)
    names, nums = extract_nums_names(text)
    res = dict(zip(names, nums))
    # print(len(res))
    # pprint(res)
    contacts.update(res)

working on C:\Users\simon\Documents\draft\ocr_for_contact_list\data\example_git_hub.png
(526, 936, 3)


In [5]:
print("{} contacts founds in [{}] images:".format(len(contacts), counter))
pprint(contacts)
save_dict_as_json(contacts, "contacts.json")

11 contacts founds in [1] images:
{'Commisssire San-Antonio': '01234567899',
 'Frangois Fignon home': '01234567893',
 'Frangois Fignon perso': '01234567895',
 'Frangois Fignon pro': '01234567894',
 'Hubert Bonsseur de La Bath': '01234567896',
 'Hubert perso': '01234567897',
 'Jean Claude DUSS perso': '01234567892',
 'Jean Claude DUSS pro': '01234567891',
 'Patrick Chirac': '012345678910',
 'oss': '01234567898',
 'péru': '012345678911'}


In [6]:
csv_file = 'contacts.csv'
save_dict_to_csv(csv_file, contacts)